In [1]:
import numpy as np
import pandas as pd
import collections
import matplotlib.pyplot as plt

In [2]:
movie=pd.read_csv('movie.csv')
rating=pd.read_csv('rating.csv')
tag=pd.read_csv('tag.csv')
link=pd.read_csv('link.csv')
gtag=pd.read_csv('genome_tags.csv')
gscore=pd.read_csv('genome_scores.csv')

In [3]:
tag.drop(columns='timestamp',inplace=True)
rating.drop(columns='timestamp',inplace=True)

In [4]:
g=tag[tag['tag'].isnull()]['movieId'].values.tolist()

In [5]:
tagfilter=tag[tag['tag'].isnull()].index
ratfiler=rating[rating['movieId'].isin(g)].index
linkfilter=link[link['movieId'].isin(g)].index
moviefilter=movie[movie['movieId'].isin(g)].index
gscorefilter=gscore[gscore['movieId'].isin(g)].index

In [6]:
tag.drop(tagfilter,inplace=True)
rating.drop(ratfiler,inplace=True)
link.drop(linkfilter,inplace=True)
movie.drop(moviefilter,inplace=True)
gscore.drop(gscorefilter,inplace=True)

In [7]:
x=rating['userId'].value_counts()>350
filter_user=x[x].index.tolist()
rating=rating[rating['userId'].isin(filter_user)]

In [8]:
average_rating = rating[['movieId','rating']].groupby('movieId', as_index=False).mean()

In [9]:
movie_avg_rating = movie.merge(average_rating, on='movieId', how='inner')

In [10]:
movie_avg_rating

,movieId,title,genres,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.855256
1,2,Jumanji (1995),Adventure|Children|Fantasy,2.970250
2,3,Grumpier Old Men (1995),Comedy|Romance,2.888781
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.635166
4,5,Father of the Bride Part II (1995),Comedy,2.720454
...,...,...,...,...
26265,131254,Kein Bund für's Leben (2007),Comedy,4.000000
26266,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,4.000000
26267,131258,The Pirates (2014),Adventure,2.500000
26268,131260,Rentun Ruusu (2001),(no genres listed),3.000000


In [11]:
def splitDataFrameList(df,target_column,separator):
    ''' df = dataframe to split,
    target_column = the column containing the values to split
    separator = the symbol used to perform the split
    returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
    The values in the other columns are duplicated across the newly divided rows.
    '''
    def splitListToRows(row,row_accumulator,target_column,separator):
        split_row = row[target_column].split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
    new_df = pd.DataFrame(new_rows)
    return new_df

splitDataFrameList(movie_avg_rating,"genres","|").head(10)

,movieId,title,genres,rating
0,1,Toy Story (1995),Adventure,3.855256
1,1,Toy Story (1995),Animation,3.855256
2,1,Toy Story (1995),Children,3.855256
3,1,Toy Story (1995),Comedy,3.855256
4,1,Toy Story (1995),Fantasy,3.855256
5,2,Jumanji (1995),Adventure,2.970250
6,2,Jumanji (1995),Children,2.970250
7,2,Jumanji (1995),Fantasy,2.970250
8,3,Grumpier Old Men (1995),Comedy,2.888781
9,3,Grumpier Old Men (1995),Romance,2.888781


In [12]:
movies_with_genres=splitDataFrameList(movie_avg_rating,"genres","|")

In [13]:
movies_with_genres.isnull().sum()

movieId    0
title      0
genres     0
rating     0
dtype: int64

In [14]:
ptgenre=movies_with_genres.pivot_table(index='title',columns='genres',values='rating')

In [15]:
ptgenre.fillna(-1,inplace=True)

In [16]:
ptgenre

genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
title,,,,,,,,,,,,,,,,,,,,
#chicagoGirl: The Social Network Takes on a Dictator (2013),-1.0,-1.0,-1.0,-1.000000,-1.0,-1.000000,-1.000000,3.000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0
$ (Dollars) (1971),-1.0,-1.0,-1.0,-1.000000,-1.0,3.111111,3.111111,-1.000,3.111111,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0
$5 a Day (2008),-1.0,-1.0,-1.0,-1.000000,-1.0,3.038462,-1.000000,-1.000,3.038462,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0
$9.99 (2008),-1.0,-1.0,-1.0,2.913043,-1.0,-1.000000,-1.000000,-1.000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0
$ellebrity (Sellebrity) (2012),-1.0,-1.0,-1.0,-1.000000,-1.0,-1.000000,-1.000000,3.500,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
À nos amours (1983),-1.0,-1.0,-1.0,-1.000000,-1.0,-1.000000,-1.000000,-1.000,3.289474,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,3.289474,-1.0,-1.0,-1.0,-1.0
À nous la liberté (Freedom for Us) (1931),-1.0,-1.0,-1.0,-1.000000,-1.0,3.604651,-1.000000,-1.000,-1.000000,-1.0,-1.0,-1.0,-1.0,3.604651,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0
À propos de Nice (1930),-1.0,-1.0,-1.0,-1.000000,-1.0,-1.000000,-1.000000,3.125,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0


In [17]:
from scipy.sparse import csr_matrix
matrix_genre=csr_matrix(ptgenre)

In [18]:
matrix_genre

<26256x20 sparse matrix of type '<class 'numpy.float64'>'
	with 525120 stored elements in Compressed Sparse Row format>

In [19]:
from sklearn.neighbors import NearestNeighbors
NN_genre=NearestNeighbors(algorithm='brute')

In [20]:
NN_genre.fit(matrix_genre)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [21]:
def genre_movie_recommender(movie):
    d_genre,s_genre=NN_genre.kneighbors(ptgenre[ptgenre.index==movie].values.reshape(1,-1),n_neighbors=10)
    for i in s_genre[0]:
        print(ptgenre.index[i])

# GENRE BASED RECOMMENDER 

In [22]:
genre_movie_recommender('Toy Story (1995)')

Toy Story (1995)
Monsters, Inc. (2001)
Toy Story 2 (1999)
Brother Bear 2 (2006)
Boxtrolls, The (2014)
Emperor's New Groove, The (2000)
Toy Story Toons: Hawaiian Vacation (2011)
Antz (1998)
DuckTales: The Movie - Treasure of the Lost Lamp (1990)
Asterix and the Vikings (Astérix et les Vikings) (2006)


In [23]:
movie_user = movie.merge(rating, on='movieId', how='inner')

In [24]:
movie_user

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.5
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,24,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,54,4.0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,58,5.0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,91,4.0
...,...,...,...,...,...
8821030,131254,Kein Bund für's Leben (2007),Comedy,79570,4.0
8821031,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,79570,4.0
8821032,131258,The Pirates (2014),Adventure,28906,2.5
8821033,131260,Rentun Ruusu (2001),(no genres listed),65409,3.0


In [25]:
ptuser=movie_user.pivot_table(index='title',columns='userId',values='rating')

In [26]:
ptuser.fillna(-1,inplace=True)

In [27]:
ptuser

userId,11,24,54,58,91,96,104,116,131,134,...,138404,138406,138411,138414,138437,138454,138456,138472,138474,138493
title,,,,,,,,,,,,,,,,,,,,,
#chicagoGirl: The Social Network Takes on a Dictator (2013),-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
$ (Dollars) (1971),-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
$5 a Day (2008),-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
$9.99 (2008),-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
$ellebrity (Sellebrity) (2012),-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
À nos amours (1983),-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
À nous la liberté (Freedom for Us) (1931),-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,3.5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
À propos de Nice (1930),-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [28]:
from scipy.sparse import csr_matrix
matrix_rating=csr_matrix(ptuser)

In [29]:
matrix_rating

<26256x12926 sparse matrix of type '<class 'numpy.float64'>'
	with 339385056 stored elements in Compressed Sparse Row format>

In [30]:
from sklearn.neighbors import NearestNeighbors
NN_rating=NearestNeighbors(algorithm='auto')

In [32]:
NN_rating.fit(matrix_rating)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [33]:
def movie_recommender_user(movie):
    d_user,s_user=NN_rating.kneighbors(ptuser[ptuser.index==movie].values.reshape(1,-1),n_neighbors=10)
    for i in s_user[0]:
        print(ptuser.index[i])

# User Rating Based Movie Recommendation

In [34]:
movie_recommender_user("Harry Potter and the Chamber of Secrets (2002)")

Harry Potter and the Chamber of Secrets (2002)
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Harry Potter and the Prisoner of Azkaban (2004)
Harry Potter and the Goblet of Fire (2005)
Star Wars: Episode II - Attack of the Clones (2002)
Harry Potter and the Order of the Phoenix (2007)
Men in Black II (a.k.a. MIIB) (a.k.a. MIB 2) (2002)
Shrek 2 (2004)
Matrix Reloaded, The (2003)
X2: X-Men United (2003)


In [35]:
gscore=gscore[gscore['relevance']>0.7]

In [36]:
gscore=gscore.merge(gtag,on='tagId',how='inner')

In [37]:
gscore['movieId'].nunique()

10328

In [38]:
gscore=gscore.merge(movie,on='movieId',how='inner')

In [39]:
gscore

,movieId,tagId,relevance,tag,title,genres
0,1,29,0.89200,adventure,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,63,0.93325,animated,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,1,64,0.98575,animation,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,1,186,0.95650,cartoon,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,1,193,0.81925,cgi,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...,...
182671,7165,283,0.98850,dance,"Company, The (2003)",Drama|Musical
182672,7165,284,0.97200,dancing,"Company, The (2003)",Drama|Musical
182673,7165,104,0.81825,ballet,"Company, The (2003)",Drama|Musical
182674,6613,318,0.73875,dolphins,"Day of the Dolphin, The (1973)",Drama


In [40]:
gscore.drop(columns='genres',inplace=True)

In [41]:
pttags=gscore.pivot_table(index='title',columns='tag',values='relevance')

In [42]:
pttags.fillna(-1,inplace=True)

In [43]:
pttags

tag,007,007 (series),18th century,1920s,1930s,1950s,1960s,1970s,1980s,19th century,...,world politics,world war i,world war ii,writer's life,writers,writing,wuxia,wwii,zombie,zombies
title,,,,,,,,,,,,,,,,,,,,,
$9.99 (2008),-1.00000,-1.00000,-1.0,-1.0,-1.00000,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
'71 (2014),-1.00000,-1.00000,-1.0,-1.0,-1.00000,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
'Round Midnight (1986),-1.00000,-1.00000,-1.0,-1.0,-1.00000,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
'Til There Was You (1997),-1.00000,-1.00000,-1.0,-1.0,-1.00000,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
"'burbs, The (1989)",-1.00000,-1.00000,-1.0,-1.0,-1.00000,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ (1999),-1.00000,-1.00000,-1.0,-1.0,-1.00000,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
xXx (2002),0.90675,0.95225,-1.0,-1.0,-1.00000,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
xXx: State of the Union (2005),-1.00000,-1.00000,-1.0,-1.0,-1.00000,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [44]:
from scipy.sparse import csr_matrix
matrix_tags=csr_matrix(pttags)

In [45]:
matrix_tags

<10327x1121 sparse matrix of type '<class 'numpy.float64'>'
	with 11576567 stored elements in Compressed Sparse Row format>

In [46]:
from sklearn.neighbors import NearestNeighbors
NN_tags=NearestNeighbors(algorithm='brute')

In [47]:
NN_tags.fit(matrix_tags)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [48]:
def movie_recommender_tags(movie):
    d_tags,s_tags=NN_tags.kneighbors(pttags[pttags.index==movie].values.reshape(1,-1),n_neighbors=10)
    for i in s_tags[0]:
        print(pttags.index[i])

# Tags Based Movie Recommendation


In [49]:
movie_recommender_tags('Harry Potter and the Chamber of Secrets (2002)')

Harry Potter and the Chamber of Secrets (2002)
Harry Potter and the Prisoner of Azkaban (2004)
Harry Potter and the Goblet of Fire (2005)
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Harry Potter and the Order of the Phoenix (2007)
Harry Potter and the Half-Blood Prince (2009)
Harry Potter and the Deathly Hallows: Part 1 (2010)
Spiderwick Chronicles, The (2008)
Seeker: The Dark Is Rising, The (2007)
Chronicles of Narnia: The Lion, the Witch and the Wardrobe, The (2005)
